In [13]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("data/financials_raw.csv", encoding="ISO-8859-1")

/var/folders/wk/s65gzdnd2zq9zgz4swg1smsw0000gn/T/ipykernel_97735/4115356464.py:4: DtypeWarning: Columns (3,4,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/financials.csv", encoding="ISO-8859-1")


In [14]:
# Display the column names
print(df.columns)

Index(['ENTITY_CURRENCY', 'CURRENCY', 'EXCHANGE_RATE', 'SOLD_TO_CITY',
       'SOLD_TO_COUNTRY', 'INVOICE_DATE', 'INVOICE_YEAR', 'INVOICE_MONTH',
       'UNIT_OF_MEASURE', 'INV_LINE_SEQ__CRM_USAGE_ONLY_', 'ITEM_CODE',
       'ITEM_EU_FAMILY', 'IS_INTERCO_FLAG', 'INVOICED_QUANTITY',
       'INVOICE_LIST_PRICE', 'INVOICE_PRICE', 'INVOICE_STD_COST',
       'ITEM_MATERIAL_COST', 'ITEM_LABOR_COST', 'ITEM_BURDEN_COST',
       'ITEM_OVERHEAD_COST', 'ITEM_SUBCONTRACTING_COST', 'ITEM_STANDARD_COST',
       'SALES_FUNCTIONAL_CURRENCY', 'COGS', 'GROSS_AMOUNT', 'DISCOUNT_AMOUNT',
       'FY', 'Financial Period', 'FY Period', 'EntityID', 'InvoiceID',
       'SoldToID', 'ItemCodeID', 'Multiplicator'],
      dtype='object')


In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6439207 entries, 0 to 6439206
Data columns (total 35 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   ENTITY_CURRENCY                object 
 1   CURRENCY                       object 
 2   EXCHANGE_RATE                  float64
 3   SOLD_TO_CITY                   object 
 4   SOLD_TO_COUNTRY                object 
 5   INVOICE_DATE                   object 
 6   INVOICE_YEAR                   int64  
 7   INVOICE_MONTH                  int64  
 8   UNIT_OF_MEASURE                object 
 9   INV_LINE_SEQ__CRM_USAGE_ONLY_  float64
 10  ITEM_CODE                      object 
 11  ITEM_EU_FAMILY                 float64
 12  IS_INTERCO_FLAG                object 
 13  INVOICED_QUANTITY              float64
 14  INVOICE_LIST_PRICE             float64
 15  INVOICE_PRICE                  float64
 16  INVOICE_STD_COST               float64
 17  ITEM_MATERIAL_COST             float64
 18  IT

In [23]:
# Total GROSS_AMOUNT by INVOICE_MONTH AND INVOICE_YEAR, as int, formatted as $#,###,###.##
# Also the count of invoices for each month and year
total_gross_amount = (
    df.groupby(["INVOICE_MONTH", "INVOICE_YEAR"])
    .agg(
        GROSS_AMOUNT=("GROSS_AMOUNT", "sum"),
        INVOICE_COUNT=("INVOICED_QUANTITY", "sum"),
    )
    .reset_index()
)
total_gross_amount['GROSS_AMOUNT'] = total_gross_amount['GROSS_AMOUNT'].apply(lambda x: f"${x:,.2f}")
total_gross_amount['INVOICE_COUNT'] = total_gross_amount['INVOICE_COUNT'].astype(int)

# sort by INVOICE_YEAR and INVOICE_MONTH
total_gross_amount = total_gross_amount.sort_values(by=["INVOICE_YEAR", "INVOICE_MONTH"])
print(total_gross_amount)

    INVOICE_MONTH  INVOICE_YEAR     GROSS_AMOUNT  INVOICE_COUNT
26             11          2020       $12,585.31           2042
29             12          2020  $213,571,032.10        1769806
0               1          2021  $174,887,648.00        1370200
3               2          2021  $183,058,237.85        1055032
6               3          2021  $218,297,981.31        1132935
9               4          2021  $218,547,705.31        1013208
12              5          2021  $187,152,210.72         919287
15              6          2021       $41,608.57             21
30             12          2021  $193,484,742.68         880759
1               1          2022  $215,569,187.28         834138
4               2          2022  $204,574,156.10         828166
7               3          2022  $205,778,009.24        1003299
10              4          2022  $190,809,688.94         752260
13              5          2022  $180,731,558.45         907737
16              6          2022  $191,22